In [1]:
from datatable import dt, f, by, as_type
from datetime import datetime as d
from shapely.geometry import Point, shape, Polygon
import os, shutil, glob
import argparse
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import geopandas


/opt/conda/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
fold = '/BigData/BigData/HW1/*'
file = glob.glob(f'{fold}')
file

['/BigData/BigData/HW1/yellow_tripdata_2009-03.csv',
 '/BigData/BigData/HW1/yellow_tripdata_2009-02.csv',
 '/BigData/BigData/HW1/yellow_tripdata_2009-01.csv']

In [3]:
frame = dt.Frame()
[frame.rbind(dt.fread(f, fill=True), force=True, bynames=True) for f in file]
columns = frame.names
columns

('vendor_name',
 'Trip_Pickup_DateTime',
 'Trip_Dropoff_DateTime',
 'Passenger_Count',
 'Trip_Distance',
 'Start_Lon',
 'Start_Lat',
 'Rate_Code',
 'store_and_forward',
 'End_Lon',
 'End_Lat',
 'Payment_Type',
 'Fare_Amt',
 'surcharge',
 'mta_tax',
 'Tip_Amt',
 'Tolls_Amt',
 'Total_Amt')

In [4]:
'''
Delete all NA row column and row
'''
del frame[:, 'store_and_forward']
del frame[:, 'Rate_Code']
del frame[:, 'mta_tax']
del frame[dt.rowall(f[:]==None),:]

In [7]:

frame[:,dt.update(Trip_Pickup_DateTime = dt.time.hour(f.Trip_Pickup_DateTime), \
                    Trip_Dropoff_DateTime = dt.time.hour(f.Trip_Dropoff_DateTime))]
frame

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,Tip_Amt,Tolls_Amt,Total_Amt
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CMT,15,15,1,0.3,−73.9707,40.7964,−73.9736,40.7921,Cash,4.1,0,0,0,4.1
1,CMT,0,0,1,1.6,−74.0073,40.74,−74.0047,40.7519,Cash,7,0,0,0,7
2,DDS,19,20,1,2,−73.9764,40.7567,−73.9544,40.7672,CASH,8.1,0.5,0,0,8.6
3,CMT,22,22,3,1.6,−73.9906,40.7515,−74.0084,40.7466,Cash,7,0,0,0,7
4,CMT,22,23,2,3.8,−73.9938,40.7342,−73.9588,40.7699,Cash,12.6,0,0,0,12.6
5,CMT,1,1,1,0.7,−73.9674,40.7809,−73.9681,40.7719,Cash,5.4,0,0,0,5.4
6,CMT,22,22,1,0,−73.9616,40.7565,−73.9616,40.7566,No Charge,3,0,0,0,3
7,CMT,7,7,1,0.9,−73.9338,40.854,−73.9396,40.8414,Cash,4.5,0,0,0,4.5
8,CMT,13,13,1,1.1,−73.9645,40.7704,−73.9819,40.7737,Cash,5.7,0,0,0,5.7


In [8]:
from pyproj import CRS
crs = CRS('epsg:4326')
def cor2loc(df, lon_var, lat_var, name_var):
    local_df = df.copy()
    shape_df = geopandas.read_file('taxi_zones/taxi_zones.shp')
    shape_df.drop(['OBJECTID', 'Shape_Area', 'Shape_Leng', 'borough'], axis=1, inplace=True)
    shape_df = shape_df.to_crs(crs)
    
    try:
        local_gdf = geopandas.GeoDataFrame(local_df, crs=crs,
                                          geometry=[Point(xy) for xy in zip(local_df[lon_var], local_df[lat_var])])
        local_gdf = geopandas.sjoin(local_gdf, shape_df, how='left', op='within')
        local_gdf = local_gdf.drop('geometry', axis=1)
        local_gdf = local_gdf.drop('index_right', axis=1)
        local_gdf.rename(columns={'zone':f'{name_var}_Zone', 'LocationID':f'{name_var}_ID'}, inplace=True)
        return local_gdf
    except ValueError as ve:
        print(ve)
        print(ve.stacktrace())
        series = df[lon_var]
        return series

In [10]:
# Partition dataframe to multiple chunk
num_of_pool = 16
frame = frame.to_pandas()
n = frame.shape[0] // num_of_pool
list_frame = [frame[i:i+n] for i in range(0, frame.shape[0],n)]

In [12]:
# Mapping Lon and Lat to zone and location ID
from multiprocessing import Pool
from itertools import repeat
def multiple_mapping(num_of_pool, list_df,lon_var, lat_var, name_var):
    with Pool(num_of_pool) as pool:
        second_arg = lon_var
        third_arg = lat_var
        forth_arg = name_var
        new_list = pool.starmap(cor2loc, zip(list_df, repeat(second_arg), repeat(third_arg), repeat(forth_arg)))
    return new_list

In [13]:
list_frame = multiple_mapping(num_of_pool, list_frame, 'Start_Lon', 'Start_Lat', 'pick_up')
list_frame = multiple_mapping(num_of_pool, list_frame, 'End_Lon', 'End_Lat', 'drop_off')

/opt/conda/lib/python3.8/multiprocessing/pool.py:125: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = (True, func(*args, **kwds))
/opt/conda/lib/python3.8/multiprocessing/pool.py:125: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = (True, func(*args, **kwds))
/opt/conda/lib/python3.8/multiprocessing/pool.py:125: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = (True, func(*args, **kwds))
/opt/conda/lib/python3.8/multiprocessing/pool.py:125: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = (True, func(*args, **kwds))
/opt/conda/lib/python3.8/multiprocessing/pool.py:125: FutureWarning: The `op` parameter 

[        vendor_name  Trip_Pickup_DateTime  Trip_Dropoff_DateTime  \
 0               CMT                    15                     15   
 1               CMT                     0                      0   
 2               DDS                    19                     20   
 3               CMT                    22                     22   
 4               CMT                    22                     23   
 ...             ...                   ...                    ...   
 2616239         CMT                    21                     22   
 2616240         DDS                    21                     21   
 2616241         CMT                    20                     20   
 2616242         CMT                    18                     18   
 2616243         CMT                    17                     17   
 
          Passenger_Count  Trip_Distance  Start_Lon  Start_Lat    End_Lon  \
 0                      1            0.3 -73.970709  40.796382 -73.973602   
 1              

In [15]:
new_frame = pd.concat(list_frame)
new_frame.to_csv('Finish_mapping.csv')

In [17]:
new_frame

TypeError: Unknown type for the first argument in fread: <class 'geopandas.geodataframe.GeoDataFrame'>

In [18]:
frame = dt.fread('Finish_mapping.csv')

In [19]:
frame

,C0,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat,…,Total_Amt,pick_up_Zone,pick_up_ID,drop_off_Zone,drop_off_ID
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,CMT,15,15,1,0.3,−73.9707,40.7964,−73.9736,40.7921,…,4.1,Manhattan Valley,151,Upper West Side North,238
1,1,CMT,0,0,1,1.6,−74.0073,40.74,−74.0047,40.7519,…,7,Meatpacking/West Village West,158,West Chelsea/Hudson Yards,246
2,2,DDS,19,20,1,2,−73.9764,40.7567,−73.9544,40.7672,…,8.6,Midtown Center,161,Lenox Hill East,140
3,3,CMT,22,22,3,1.6,−73.9906,40.7515,−74.0084,40.7466,…,7,Garment District,100,West Chelsea/Hudson Yards,246
4,4,CMT,22,23,2,3.8,−73.9938,40.7342,−73.9588,40.7699,…,12.6,Greenwich Village North,113,Lenox Hill West,141
5,5,CMT,1,1,1,0.7,−73.9674,40.7809,−73.9681,40.7719,…,5.4,Central Park,43,Central Park,43
6,6,CMT,22,22,1,0,−73.9616,40.7565,−73.9616,40.7566,…,3,Sutton Place/Turtle Bay North,229,Sutton Place/Turtle Bay North,229
7,7,CMT,7,7,1,0.9,−73.9338,40.854,−73.9396,40.8414,…,4.5,Washington Heights North,243,Washington Heights South,244
8,8,CMT,13,13,1,1.1,−73.9645,40.7704,−73.9819,40.7737,…,5.7,Upper East Side South,237,Lincoln Square East,142


In [23]:
frame[:,dt.count(), by('pick_up_Zone')].sort(-f.count)

,pick_up_Zone,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Times Sq/Theatre District,1581229
1,Midtown Center,1558366
2,Upper East Side South,1546520
3,East Village,1503926
4,Upper East Side North,1424176
5,Midtown East,1401322
6,Murray Hill,1396157
7,Union Sq,1349435
8,Clinton East,1261417


In [24]:
frame[:,dt.count(), by('drop_off_Zone')].sort(-f.count)

,drop_off_Zone,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Midtown Center,1673651
1,Upper East Side North,1446570
2,Times Sq/Theatre District,1438987
3,Upper East Side South,1402008
4,Murray Hill,1368418
5,Midtown East,1301625
6,Union Sq,1207092
7,East Village,1197367
8,Lincoln Square East,1127789


In [26]:
frame[:, dt.count(), by('pick_up_Zone', 'drop_off_Zone')].sort(-f.count)

,pick_up_Zone,drop_off_Zone,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,,,757214
1,Upper East Side South,Upper East Side North,185650
2,Upper East Side North,Upper East Side North,172571
3,Upper East Side North,Upper East Side South,166715
4,Upper East Side South,Upper East Side South,157334
5,East Village,East Village,107938
6,Upper West Side South,Lincoln Square East,105543
7,Upper West Side South,Upper West Side North,101245
8,Upper East Side South,Midtown Center,97364


In [27]:
frame[:, dt.count(), by('Trip_Pickup_DateTime')]

,Trip_Pickup_DateTime,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,1621426
1,1,1191211
2,2,894354
3,3,674986
4,4,488542
5,5,379852
6,6,804853
7,7,1477144
8,8,1941097


In [29]:
frame[:, dt.count(), by('Trip_Dropoff_DateTime')]

,Trip_Dropoff_DateTime,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,1707622
1,1,1264955
2,2,942516
3,3,702340
4,4,535283
5,5,373470
6,6,714493
7,7,1342909
8,8,1850614
